YOLOV3 AI Object detection project

In [1]:
pip install opencv-python

In [2]:
pip install tensorflow

In [ ]:
pip install --upgrade pip

In [4]:
import cv2
import numpy as np
import tensorflow as tf  # for YOLO v3

print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"TensorFlow version: {tf.__version__}")  # for YOLO v3



OpenCV version: 4.8.0
NumPy version: 1.24.3
TensorFlow version: 2.13.0


#Load YOLO v3 model and class labels: 

In [5]:
def load_model():
    # Load the pre-trained YOLO weights and configuration
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    
    #Load the class labels 
    with open ("./coco.names.txt", "r")as f:
        classes = [line.strip() for line in f.readlines()]
    return net, classes 
 



In [6]:
classes = []
boxes = []
confidences = []
image_path = './person.jpg'
image = cv2.imread(image_path)

In [7]:
def perform_object_detection(image_path):
    #Load the YOLO model and class labels
    net, classes = load_model()
    
    # Image path 
    image_path = './person.jpg'
    #Load the image
    image = cv2.imread(image_path)
    

    #Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 1/255, (416,416), (0,0,0), True, crop = False)
    
    #Set the input to the network 
    net.setInput(blob)
    
    # Forward pass through the network
    layer_names = net.getLayerNames()
    output_layers = net.getUnconnectedOutLayersNames()
    outputs = net.forward(output_layers)

    # Process the outputs
    boxes = []
    confidences = []
    class_ids = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id] 

            # Filter detections by confidence threshold
            if confidence > 0.5: 
                # Scale the bounding box coordinates to the original image size
                width = image.shape[1]
                height = image.shape[0]
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Store the box, confidence, and class ID
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
         # Apply non-maximum suppression to remove redundant overlapping boxes
        indexes = cv2.dnn.NMSBoxes(boxes,confidences, 0.5, 0.4)

        # Draw bounding boxes and labels on the image
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = classes[class_ids[i]]
                confidence = confidences[i]
                color = colors[class_ids[i]]
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(image, f"{label}: {confidence:.2f}", (x, y - 5), font, 0.5, color, 2) 
                #maintain the position of the bounding boxes in respect to the images that are being entered 

        # Display the image with bounding boxes
    cv2.imshow("Object Detection", image)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

                

In [8]:
# Test the object detection
# Replace with the path to your image
perform_object_detection(image_path)